# RAG from PDF files

This demo provides an example where retrieval is performed on PDF files.

Note: To run the code with your own data, simply update the folder path (or the `folder_path` variable) accordingly.

In [ ]:
%load_ext autoreload
%autoreload 2

### Import libraries

In [ ]:
import os
from src.PDFProcessor import PDFProcessor
from src.LargeLanguageModel import LargeLanguageModel
from src.IndexManager import IndexManager

### Set paths

In [ ]:
folder_path = "pdf_files/"
index_path = 'resources/faiss_embeddings.index'

### Read PDF files from folder  
Extract text from PDF files in the specified folder (e.g., `folder_path = "pdf_files/"`)

In [ ]:
texts, info = PDFProcessor.extract_text_from_pdfs_in_folder(folder_path)

### Split into smaller chunks for better retrieval

In [ ]:
chunks, info = IndexManager.chunk_texts_and_info(texts, info)

### Create or Load the Embedding Index

Prepare dataset of embeddings from text for the retrieval. Build an index for efficient similarity search.

Find relevant text and then use an LLM to generate a response based on this text.

In [ ]:
# Initialize LLM and IndexManager
llm = LargeLanguageModel(model_id="meta-llama/Llama-3.2-3B-Instruct")
index_manager = IndexManager(chunks, info, llm)

In [ ]:
if not os.path.isfile(index_path):
    print("Create embedding index")
    index = index_manager.create_index(texts,
                                       index_path)
else:
    print("Load embedding index")
    index = index_manager.load_index(index_path)

### Query PDF documents

##### Retrieval

Retrieves the top-k relevant references, and constructs a prompt for the LLM

In [ ]:
query = "What does this document say about attention mechanism?"
references = index_manager.query(text=query, top_k=5)

context = "\n".join([text for text, _ in references])
prompt = f"Context:\n{context}\n\nQuestion: {query}\nAnswer:"

##### Generate answer

In [ ]:
answer = llm.decode(prompt)

print("Answer:")
print(answer)
print("\nReferences:")
for text_cur, info_cur in references:
    fname, page_num = info_cur.split()
    print(f"- {fname}, page {page_num}: {text_cur[:20]}")